In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import re
import requests
import json
import os


In [8]:
# read in all gene names
gene_list_raw =pd.read_csv("230323_EST_ENSG_GENE_new.csv")

gene_list_raw

,index,ENST,ENSG,protein,gene,Uniprot
0,0,ENST00000646641,ENSG00000267534,s1pr2_human,S1PR2,O95136
1,121,ENST00000547270,ENSG00000257138,t2r38_human,TAS2R38,P59533
2,219,ENST00000390675,ENSG00000256436,t2r31_human,TAS2R31,P59538
3,344,ENST00000539585,ENSG00000256188,t2r30_human,TAS2R30,P59541
4,483,ENST00000538986,ENSG00000255837,t2r20_human,TAS2R20,P59543
...,...,...,...,...,...,...
395,64029,ENST00000510937,ENSG00000226306,npy6r_human,NPY6R,Q99463
396,64030,ENST00000641193,ENSG00000279301,o2t11_human,OR2T11,Q8NH01
397,64031,ENST00000641732,ENSG00000172146,or1a1_human,OR1A1,Q9P1Q5
398,64032,ENST00000328890,ENSG00000183024,or1g1_human,OR1G1,P47890


In [4]:
gpcr_missense = pd.read_csv('gpcr_valid_missense.csv')
gpcr_missense

,vep_input,mutation_pos,SYMBOL,Amino_acids,protein,SIFT,PolyPhen,BayesDel_addAF_score,BayesDel_noAF_score,CADD_phred,...,MutationAssessor_score,PROVEAN_score,PrimateAI_score,REVEL_score,VEST4_score,GERP++_RS,SiPhy_29way_logOdds,MutPred_score,mutation_from,mutation_to
0,ENST00000294954:p.Ala572Val,572,LHCGR,A/V,lshr_human,deleterious(0),probably_damaging(0.998),0.271259,0.151868,28.4,...,"2.685,.","-3.47,-3.51",0.647560358047,"0.613,0.613","0.778,0.789",5.68,18.7926,0.935,A,V
1,ENST00000294954:p.Ala593Pro,593,LHCGR,A/P,lshr_human,deleterious(0),probably_damaging(1),0.263129,0.14019,25.7,...,"3.28,.","-3.34,-3.56",0.664010286331,"0.595,0.595","0.897,0.938",5.68,18.7926,0.886,A,P
2,ENST00000294954:p.Asp564Gly,564,LHCGR,D/G,lshr_human,deleterious(0),probably_damaging(0.999),0.302076,0.196135,28.5,...,"3.49,.","-6.01,-5.99",0.665593266487,"0.842,0.842","0.923,0.954",5.68,15.1242,0.897,D,G
3,ENST00000294954:p.Asp578Gly,578,LHCGR,D/G,lshr_human,deleterious(0),possibly_damaging(0.685),0.294007,0.184544,27.4,...,"3.715,.","-5.96,-6.04",0.658861994743,"0.756,0.756","0.84,0.927",5.68,15.1242,0.936,D,G
4,ENST00000294954:p.Asp578His,578,LHCGR,D/H,lshr_human,deleterious(0),probably_damaging(0.952),0.275656,0.158184,24.6,...,"3.715,.","-5.94,-6.02",0.713786900043,"0.500,0.500","0.828,0.864",4.8,14.1045,0.877,D,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2480,ENST00000276198:p.Cys360Ala,360,HTR2C,C/A,5ht2c_human,deleterious(0.02),benign(0.015),-,-,-,...,-,-,-,-,-,-,-,-,C,A
2481,ENST00000276198:p.Ser361Ala,361,HTR2C,S/A,5ht2c_human,deleterious(0),probably_damaging(0.998),0.270807,0.151219,24.3,...,".,.","-2.26,-2.26",0.640748023987,"0.821,0.821","0.623,0.65",5.13,11.8568,0.886,S,A
2482,ENST00000276198:p.Tyr368Ala,368,HTR2C,Y/A,5ht2c_human,deleterious(0),probably_damaging(1),-,-,-,...,-,-,-,-,-,-,-,-,Y,A
2483,ENST00000276198:p.Tyr368Cys,368,HTR2C,Y/C,5ht2c_human,deleterious(0),probably_damaging(1),0.622314,0.656134,24.9,...,".,.","-7.09,-7.09",0.834922552109,"0.881,0.881","0.964,0.961",5.13,11.8568,0.867,Y,C


In [22]:

uniid_list = gene_list_raw[gene_list_raw['protein'].isin(np.unique(gpcr_missense['protein'].tolist()))]['Uniprot'].tolist()

uniid_list

['P35367',
 'P34972',
 'P13945',
 'P08912',
 'P21728',
 'P08172',
 'P25025',
 'P08173',
 'P28221',
 'P08908',
 'P41231',
 'P29275',
 'P35368',
 'Q7RTX0',
 'P21918',
 'Q9H244',
 'P07550',
 'P11229',
 'P32245',
 'Q14416',
 'P32246',
 'P30542',
 'P25024',
 'P51681',
 'P50406',
 'P47898',
 'P14416',
 'P49190',
 'P22888',
 'P41595',
 'P28222',
 'Q9H3N8',
 'P20309',
 'P43220',
 'P47872',
 'Q16602',
 'P48546',
 'Q03431',
 'P41597',
 'P47871',
 'P35462',
 'P28223',
 'P49682',
 'P08588',
 'P51677',
 'Q9Y5N1',
 'P34969',
 'P51685',
 'Q02643',
 'P32241',
 'P41594',
 'P41180',
 'Q13255',
 'P35348',
 'P28335',
 'P41587',
 'P61073',
 'P25021',
 'P34998',
 'P16473',
 'P23945',
 'Q13639',
 'P24530',
 'P29274',
 'P18089',
 'P0DMS8']

# for each GPCR, get AF pdb file

In [23]:
# a function to get the alphafold structure (pdb file) for each gene
def getAF(uniprot_id):
    # Set the API endpoint and the UniProt ID of the protein
    endpoint = "https://alphafold.ebi.ac.uk/files"

    # Send a GET request to the API to retrieve the protein's structure
    response = requests.get(f"{endpoint}/AF-{uniprot_id}-F1-model_v4.pdb")

    # Check the status code of the response to ensure it was successful
    if response.status_code == 200:
        filename = f"/Users/joannahench/Downloads/foldx5MacStd.tar_/GPCRdb/{uniprot_id}/{uniprot_id}.pdb"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        # Write the contents of the response to a PDB file
        with open(filename, "w") as f:
            f.write(response.text)
    else:
        print(f"Failed to retrieve structure for {uniprot_id}: {response.status_code}")



In [24]:
#get the structure
for i in range(len(uniid_list)):
    getAF(uniid_list[i])

In [26]:
unq_uniid_list = np.unique(uniid_list)
unq_uniid_list

array(['P07550', 'P08172', 'P08173', 'P08588', 'P08908', 'P08912',
       'P0DMS8', 'P11229', 'P13945', 'P14416', 'P16473', 'P18089',
       'P20309', 'P21728', 'P21918', 'P22888', 'P23945', 'P24530',
       'P25021', 'P25024', 'P25025', 'P28221', 'P28222', 'P28223',
       'P28335', 'P29274', 'P29275', 'P30542', 'P32241', 'P32245',
       'P32246', 'P34969', 'P34972', 'P34998', 'P35348', 'P35367',
       'P35368', 'P35462', 'P41180', 'P41231', 'P41587', 'P41594',
       'P41595', 'P41597', 'P43220', 'P47871', 'P47872', 'P47898',
       'P48546', 'P49190', 'P49682', 'P50406', 'P51677', 'P51681',
       'P51685', 'P61073', 'Q02643', 'Q03431', 'Q13255', 'Q13639',
       'Q14416', 'Q16602', 'Q7RTX0', 'Q9H244', 'Q9H3N8', 'Q9Y5N1'],
      dtype='<U6')

# make mutation list

In [25]:
# make the mutation file (needed input for FoldX) for each protein
uniid_list = []
# gene_list = []
for i in range(gpcr_missense.shape[0]):
    # get the uniprot id according to entryname and convert the Series to a string with index excluded
    uniprot_id = gene_list_raw.loc[gene_list_raw['protein']==gpcr_missense.iloc[i,4],'Uniprot'].to_string(index=False)
    # gene = gene_list_raw.loc[gene_list_raw['protein']==gpcr_valid_missense.iloc[i,4],'gene'].to_string(index=False)
    uniid_list.append(uniprot_id)
    # gene_list.append(gene)
    mut_info = gpcr_missense.iloc[i,30]+'A'+str(gpcr_missense.iloc[i,1])+gpcr_missense.iloc[i,31]
    with open(f"/Users/joannahench/Downloads/foldx5MacStd.tar_/GPCRdb/{uniprot_id}/individual_list_{uniprot_id}.txt", 'a') as file:
        # write a new line to the file
        file.write(mut_info+';'+"\n")
    

# make bash file

In [27]:
# generate the bash file
for i in range(len(unq_uniid_list)):
        uniprot_id = unq_uniid_list[i]
        with open("foldx_command.sh", 'a') as foldx:
                
                foldx.write(f'cd {uniprot_id}'+"\n")
                # write the command
                foldx.write(f"../../foldx --command=BuildModel --pdb={uniprot_id}.pdb --mutant-file=individual_list_{uniprot_id}.txt"+"\n")
                foldx.write('cd ..'+"\n") 

In [ ]:
../../foldx --help --command=BuildModel

In [ ]:
../../foldx --command=BuildModel --pdb=/Users/joannahench/Downloads/foldx5MacStd.tar_/GPCRdb/P0DMS8/P0DMS8.pdb --mutant-file=/Users/joannahench/Downloads/foldx5MacStd.tar_/GPCRdb/P0DMS8/individual_list_P0DMS8.txt

In [ ]:
nohup ./foldx_test.sh &